# Pythonのクラス機構

## classキーワードによるクラスの定義

In [1]:
class Page:  # クラスの定義
    def __init__(self, num, content):
        self.num = num  # ページ番号
        self.content = content  # ページの内容
    def output(self):
        return f'{self.content}'

In [2]:
Page  # クラスオブジェクトPageが定義された

__main__.Page

## インスタンスの作成

In [3]:
# インスタンス化
title_page = Page(0, 'Python Practice Book')

In [4]:
type(title_page)  # インスタンスのクラスを確認

__main__.Page

In [5]:
# Pageクラスのインスタンスか確認
isinstance(title_page, Page)

True

In [6]:
# インスタンスが持つ属性を確認
dir(title_page)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'content',
 'num',
 'output']

# インスタンス ── クラスをもとに生成されるオブジェクト

## インスタンスメソッド ── インスタンスに紐付くメソッド

In [7]:
title_page.output()  # インスタンスメソッドの呼び出し

'Python Practice Book'

#### メソッドオブジェクトと関数オブジェクト

In [8]:
class Klass:
    def some_method(self):  # インスタンスメソッドを定義
        print('method')

In [9]:
def some_function(self):  # 同じ引数の関数を定義
    print('function')

In [10]:
# 関数はfunctionクラスのインスタンス
type(some_function)

function

In [11]:
# インスタンスメソッドもfunctionクラスのインスタンス
type(Klass.some_method)

function

In [12]:
# インスタンスを通じてアクセスするとmethodクラスになる
kls = Klass()
type(kls.some_method)

method

In [13]:
# クラスオブジェクトの属性に関数を追加
Klass.some_function = some_function

In [14]:
# インスタンスメソッドとして実行
kls.some_function()

function


## インスタンス変数 ── インスタンスが保持する変数

In [15]:
title_page.section = 0
title_page.section

0

In [16]:
first_page = Page(1, 'first page')
first_page.section

AttributeError: 'Page' object has no attribute 'section'

## インスタンスの初期化

### \_\_init\_\_() ── インスタンスの初期化を行う特殊メソッド

In [17]:
# クラスの定義
class Page:
    def __init__(self, num, content, section=None):
        self.num = num
        self.content = content
        self.section = section
    def output(self):
        return f'{self.content}'

### 引数を渡してインスタンス化する

In [18]:
# インスタンスを作成
title_page = Page(0, 'Python Practice Book')

In [19]:
title_page.section  # sectionはNone

In [20]:
title_page.output()

'Python Practice Book'

In [21]:
# sectionを指定して別のインスタンスを作成
first_page = Page(1, 'first page', 1)

In [22]:
first_page.section  # sectionが指定されている

1

In [23]:
first_page.output()

'first page'

### \_\_init\_\_()と\_\_new\_\_()の違い ── イニシャライザとコンストラクタ

In [24]:
class Klass:
    def __new__(cls, *args):  # コンストラクタ
        print(f'{cls=}')
        print('new', args)
        # 作成したインスタンスを返す
        return super().__new__(cls)
    def __init__(self, *args):
        # インスタンスの初期化はこちらで行う
        print('init', args)

In [25]:
# クラスオブジェクトの呼び出し
kls = Klass(1, 2, 3)

cls=<class '__main__.Klass'>
new (1, 2, 3)
init (1, 2, 3)


### \_\_new\_\_()の注意点

In [26]:
class Evil:
    def __new__(cls, *args):
        return 1

In [27]:
# Evilクラスをインスタンス化
evil = Evil()

In [28]:
isinstance(evil, Evil)

False

In [29]:
type(evil)

int

In [30]:
# インスタンスは__new__()の戻り値
evil

1

In [31]:
class MyClass(Evil):
    def print_class(self):
        print('MyClass')

In [32]:
my = MyClass()  # myの値は1になる

In [33]:
# 追加したはずのメソッドが利用できない
my.print_class()

AttributeError: 'int' object has no attribute 'print_class'

In [34]:
my

1

## プロパティ ── インスタンスメソッドをインスタンス変数のように扱う

In [35]:
class Book:
    def __init__(self, raw_price):
        if raw_price < 0:
            ValueError('price must be positive')
        self.raw_price = raw_price
        self._discounts = 0
    @property
    def discounts(self):
        return self._discounts
    @discounts.setter
    def discounts(self, value):
        if value < 0 or 100 < value:
            raise ValueError(
                'discounts must be between 0 and 100')
        self._discounts = value
    @property
    def price(self):
        multi = 100 - self._discounts
        return int(self.raw_price * multi / 100)

In [36]:
book = Book(2000)
book.discounts  # 初期は値引率0

0

In [37]:
book.price  # 初期価格は2000

2000

In [38]:
book.discounts = 20  # 値引率を設定
book.price  # 値引き後の価格

1600

In [39]:
book.discounts = 120  # 100を超える値引率はエラー

ValueError: discounts must be between 0 and 100

### property ── 値の取得時に呼び出されるメソッド

### setter ── 値の設定時に呼び出されるメソッド

In [40]:
book.discounts = -20

ValueError: discounts must be between 0 and 100

In [41]:
book.price = 1000

AttributeError: can't set attribute

## クラスやインスタンスのプライベートな属性

## アンダースコアから始まる属性

In [42]:
book._discounts  # _で始まる変数も参照できる

20

### アンダースコア2つから始まる属性

In [43]:
class Klass:
    def __init__(self, x):
        self.__x = x

In [44]:
kls = Klass(10)
kls.__x  # この名前では参照できない

AttributeError: 'Klass' object has no attribute '__x'

In [45]:
kls._Klass__x  # 変換規則を知っていれば参照できる

10

### プライベートな属性に対するPythonコミュニティの考え方

# クラス ── インスタンスのひな型となるオブジェクト

## クラス変数 ── クラスオブジェクトが保持する変数

In [46]:
# クラス変数を持つクラスを定義
class Page:
    book_title = 'Python Practice Book'

In [47]:
Page.book_title  # インスタンスがなくても参照できる

'Python Practice Book'

In [48]:
Page.book_title = 'No title'  # クラス変数の更新
Page.book_title

'No title'

### クラス変数にはインスタンスからもアクセス可能

In [49]:
first_page = Page()
second_page = Page()

In [50]:
# クラス変数にはインスタンスからも参照可能
first_page.book_title

'No title'

In [51]:
second_page.book_title

'No title'

In [52]:
# クラス変数を更新
Page.book_title = 'Python Practice Book'

In [53]:
# クラス変数はすべてのインスタンスで共有される
first_page.book_title

'Python Practice Book'

In [54]:
second_page.book_title

'Python Practice Book'

In [55]:
# これはインスタンス変数になる
first_page.book_title = '[Draft]Python Practice Book'
first_page.book_title

'[Draft]Python Practice Book'

In [56]:
# クラス変数は変わっていない
second_page.book_title

'Python Practice Book'

In [57]:
first_page.book_title  # インスタンス変数

'[Draft]Python Practice Book'

In [58]:
# インスタンス変数を削除
del first_page.book_title

In [59]:
# インスタンスの属性にないため、クラスの属性が検索される
first_page.book_title

'Python Practice Book'

## クラスメソッド ── クラスに紐付くメソッド

In [60]:
# 属性を使ったソートに使える標準ライブラリをインポート
from operator import attrgetter

In [61]:
class Page:
    book_title = 'Python Practice Book'
    def __init__(self, num, content):
        self.num = num
        self.content = content
    def output(self):
        return f'{self.content}'
    # クラスメソッドの第1引数はクラスオブジェクト
    @classmethod
    def print_pages(cls, *pages):
        # クラスオブジェクトの利用
        print(cls.book_title)
        pages = list(pages)
        # ページ順に並べ替えて出力
        for page in sorted(pages, key=attrgetter('num')):
            print(page.output())

In [62]:
first = Page(1, 'first page')
second = Page(2, 'second page')
third = Page(3, 'third page')

In [63]:
# クラスメソッドの呼び出し
Page.print_pages(first, third, second)

Python Practice Book
first page
second page
third page


In [64]:
# インスタンスからも呼び出せる
first.print_pages(first, third, second)

Python Practice Book
first page
second page
third page


## スタティックメソッド ── 関数のように振る舞うメソッド

In [65]:
class Page:
    def __init__(self, num, content):
        self.num = num
        self.content = content
    @staticmethod  # スタティックメソッドにする
    def check_blank(page):
        return bool(page.content)

In [66]:
page = Page(1, '')
Page.check_blank(page)

False

In [67]:
def check_blank(page):  # 関数で問題ない
    return bool(page.content)

In [68]:
check_blank(page)

False

# クラスの継承

## メソッドのオーバーライドとsuper()による基底クラスへのアクセス

In [69]:
class Page:
    def __init__(self, num, content):
        self.num = num
        self.content = content
    def output(self):
        return f'{self.content}'

In [70]:
# メソッドのオーバーライド
class TitlePage(Page):
    def output(self):
        # 基底クラスのメソッドは自動では呼ばれないため
        # 明示的に呼び出す
        title = super().output()
        return title.upper()

In [71]:
title = TitlePage(0, 'Python Practice Book')
title.output()

'PYTHON PRACTICE BOOK'

## すべてのオブジェクトはobjectクラスのサブクラス

In [72]:
class Length(float):  # 組み込み型のサブクラスを作成
    def to_cm(self):
        return super().__str__() + 'cm'

In [73]:
pencil_length = Length(16)
print(pencil_length.to_cm())

16.0cm


## 多重継承 ── 複数の基底クラスを指定する

In [74]:
class HTMLPageMixin:
    def to_html(self):
        return f'<html><body>{self.output()}</body></html>'

In [75]:
# 多重継承を使ったMixinの利用
class WebPage(Page, HTMLPageMixin):
    pass

In [76]:
page = WebPage(0, 'web content')
page.to_html()

'<html><body>web content</body></html>'

### 多重継承の注意点

In [77]:
class A:
    def hello(self):
        print('Hello')

class B(A):
    def hello(self):
        print('Hola')
        super().hello()  # 基底クラスのメソッドを実行

class C(A):
    def hello(self):
        print('ニーハオ')
        super().hello()  # 基底クラスのメソッドを実行

class D(B, C):
    def hello(self):
        print('こんにちは')
        super().hello()  # 基底クラスのメソッドを実行

In [78]:
d = D()
d.hello()

こんにちは
Hola
ニーハオ
Hello


### \_\_mro\_\_属性を利用したメソッド解決順序の確認

In [79]:
D.__mro__  # メソッド解決順序を確認

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [80]:
d.hello()

こんにちは
Hola
ニーハオ
Hello


# 本章のまとめ